In [0]:
# Connect Google - Drive

from google.colab import drive
drive.mount('/content/gdrive')
!ln -s "/content/gdrive/My Drive" "/content/mydrive"

In [0]:
from google.colab import files

In [0]:
# Downloading the image dataset from Dropbox with !wget

import os
os.chdir("/content")

!wget https://www.dropbox.com/s/sj686ss02d3mqlx/image_classfier.tar.gz

# Uncompress zipped file
!tar -xzvf image_classfier.tar.gz -C /content

In [0]:
# Tensorboard

!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip -o ngrok-stable-linux-amd64.zip

LOG_DIR = '/content/gdrive/My Drive/image_classifier/log'
get_ipython().system_raw(
    'tensorboard --logdir {} --host 0.0.0.0 --port 8080 &'
    .format(LOG_DIR)
)

get_ipython().system_raw('./ngrok http 8080 &')
! curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

In [0]:
# Load model/confusion matrix

# Importing the required libraries

import matplotlib.pyplot as plt
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from keras.models import load_model
import pandas as pd
from numpy import argmax

# Confusion matrix

def plot_confusion_matrix(cm, target_names,title='Confusion matrix',cmap=None, normalize=False):
  accuracy = np.trace(cm) / float(np.sum(cm))
  misclass = 1 - accuracy
  if cmap is None:
      cmap = plt.get_cmap('Blues')
  plt.figure(figsize=(10, 8))
  plt.imshow(cm, interpolation='nearest', cmap=cmap)
  plt.title(title)
  plt.colorbar()

  if target_names is not None:
      tick_marks = np.arange(len(target_names))
      plt.xticks(tick_marks, target_names, rotation=45)
      plt.yticks(tick_marks, target_names)

  if normalize:
      cm = cm.astype('float32') / cm.sum(axis=1)
      cm = np.round(cm,2)
        

  thresh = cm.max() / 1.5 if normalize else cm.max() / 2
  for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
      if normalize:
          plt.text(j, i, "{:0.2f}".format(cm[i, j]),
                   horizontalalignment="center",
                   color="white" if cm[i, j] > thresh else "black")
      else:
          plt.text(j, i, "{:,}".format(cm[i, j]),
                   horizontalalignment="center",
                   color="white" if cm[i, j] > thresh else "black")

  plt.tight_layout()
  plt.ylabel('True label')
  plt.xlabel("Predicted label\naccuracy={:0.4f}\n misclass={:0.4f}".format(accuracy, misclass))
  plt.savefig('/content/mydrive/image_classifier/cm.jpg') 
  plt.show() 

filepath = '/content/mydrive/image_classifier/weights/weights_vgg16_scratch2.h5'


model_trained = load_model(filepath)

y_pred = model_trained.predict(X_test) # predicted label
y_pred = np.argmax(y_pred, axis=1)

y_true = np.argmax(y_test, axis=1) # true label

confusion_mtx = confusion_matrix(y_true, y_pred)

#----------------------------------------------------------------------------------------------

# plot the confusion matrix

plot_confusion_matrix(confusion_mtx, normalize=False, target_names=pd.unique(labelList))
print('')

#-----------------------------------------------------------------------------------------------

# plot classifcation report

print(classification_report(y_true, y_pred, target_names=pd.unique(labelList)))
print('')

In [0]:
# Testing on random images

# Importing the required libraries

from numpy import argmax
import matplotlib.pyplot as plt
from keras.preprocessing.image import load_img
import pandas as pd
from keras.models import load_model

# Random input image prediction

target_names = pd.unique(labelList)
filepath = '/content/mydrive/image_classifier/weights/weights_vgg16_scratch2.h5'
model_trained = load_model(filepath)

def predict_random(path):   
    # image pre_processing 
    img_t = load_img(path)
    # show image
    plt.figure(figsize=(5,5))
    plt.imshow(img_t)
    plt.axis('off')
    plt.show()
    
    img_t = img_t.resize((imgSize,imgSize))
    img_t = np.array(img_t)
    img_t = img_t/255.0
    
    probs = model_trained.predict(np.expand_dims(img_t, axis=0))
    
    for idx in probs.argsort()[0][::-1][:5]:
      print("{:.2f}%".format(probs[0][idx]*100), "\t", target_names[idx].split("-")[-1])


# Random images

predict_random('/content/gdrive/My Drive/image_classifier/random/rn_car.jpg')
predict_random('/content/gdrive/My Drive/image_classifier/random/rn_car2.jpg')
predict_random('/content/gdrive/My Drive/image_classifier/random/rn_cross2.jpg')
predict_random('/content/gdrive/My Drive/image_classifier/random/rn_cross3.jpg')
predict_random('/content/gdrive/My Drive/image_classifier/random/rn_human.jpg')
predict_random('/content/gdrive/My Drive/image_classifier/random/rn_person2.jpg')
predict_random('/content/gdrive/My Drive/image_classifier/random/rn_person3.jpg')
predict_random('/content/gdrive/My Drive/image_classifier/random/rn_person4.jpg')
predict_random('/content/gdrive/My Drive/image_classifier/random/rn_person5.jpg')
predict_random('/content/gdrive/My Drive/image_classifier/random/rn_person6.jpg')
predict_random('/content/gdrive/My Drive/image_classifier/random/rn_tram.jpg')
predict_random('/content/gdrive/My Drive/image_classifier/random/rn_tram2.jpg')
predict_random('/content/gdrive/My Drive/image_classifier/random/rn_tram3.jpg')
predict_random('/content/gdrive/My Drive/image_classifier/random/rn_car3.jpg')
predict_random('/content/gdrive/My Drive/image_classifier/random/rn_tram4.jpg')
predict_random('/content/gdrive/My Drive/image_classifier/random/rn_zebra.jpg')
predict_random('/content/gdrive/My Drive/image_classifier/random/rn_zebra2.jpg')

predict_random('/content/gdrive/My Drive/image_classifier/random/rn_person7.jpg')
predict_random('/content/gdrive/My Drive/image_classifier/random/rn_tram5.jpg')

predict_random('/content/gdrive/My Drive/image_classifier/random/rn_person8.jpg')


In [0]:
# Image calssifier

# Importing the required libraries

import os,shutil,math,scipy,cv2
from tqdm import tqdm
import cv2
from skimage.io import imread
import numpy as np
from sklearn.preprocessing import LabelEncoder
from keras.utils.np_utils import to_categorical
from sklearn.model_selection import train_test_split
from numpy import argmax
from keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
import random as rn
from keras.models import Sequential
from keras.layers.convolutional import Conv2D, MaxPooling2D 
from keras.layers import Flatten, Dropout, Dense, GlobalAveragePooling2D, BatchNormalization, Activation
from keras.optimizers import Adam, RMSprop, SGD
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau, TensorBoard
import itertools
from keras.applications.vgg16 import VGG16
from keras.applications.vgg19 import VGG19
from keras.applications.resnet import ResNet50
from keras.preprocessing.image import load_img
import pandas as pd
import tensorflow as tf

# ------------------------------------------------------------------------------------

# Showing model final history

def finalHistory(history):
	
  fig, axis = plt.subplots(1,2, figsize=(15,5))
  accuracy = history.history['acc']
  val_accuracy = history.history['val_acc']
  loss = history.history['loss']
  val_loss = history.history['val_loss']
  epochs = history.epoch
	 
  axis[0].set_title('Training and validation loss')
  axis[0].plot(epochs, loss, 'r', label = 'Training loss')
  axis[0].plot(epochs, val_loss, 'b', label = 'Validation loss', linestyle='--')
  axis[0].set_xlabel('epochs')
  axis[0].set_ylabel('loss')
  
  axis[1].set_title('Training and validation accuracy')
  axis[1].plot(epochs, accuracy, 'r', label = 'Training accuracy')
  axis[1].plot(epochs, val_accuracy, 'b', label = 'Validation accuracy', linestyle='--')
  axis[1].set_xlabel('epochs')
  axis[1].set_ylabel('accuracy')
  
  axis[0].legend()
  axis[1].legend()

  plt.savefig('/content/mydrive/image_classifier/saved/history_feature_e_256_5.png')

#-----------------------------------------------------------------------------------

# Read in the data

imgSize = 150
batchSize = 128

imgList = [] # empty image list
labelList = [] # empty label list

def label_assignment(img, label):
	return label

def trainingData(label, objectDirs): # training data function, input the label and te object dictionaries
	
  for img in tqdm(os.listdir(objectDirs)): # using tqdm for listing the images in the dctionaries
    
    path = os.path.join(objectDirs, img) # join the dictionary with the right image
    img = cv2.imread(path,cv2.IMREAD_COLOR) # read in the path in rgb colours
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = cv2.resize(img,(imgSize,imgSize)) # resize the images for a specified size (150,150)  
    label = label_assignment(img, label)
    imgList.append(np.array(img)) # transform images into numpy array and add them to the empty image_list
    labelList.append(str(label)) # transform the labels into string and add them to the empty label_list
    
#---------------------------------------------------------------------------------------

# Dataset libraries

carDir = '/content/image_classfier/car'
crossing_signDir = '/content/image_classfier/crossing_sign'
personDir = '/content/image_classfier/person'
tramDir = '/content/image_classfier/tram'
zebraDir = '/content/image_classfier/zebra'

weights_path = '/content/mydrive/image_classifier/transfer_weights/vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5'

trainingData('car', carDir)
trainingData('crossing sign', crossing_signDir)
trainingData('person', personDir)
trainingData('tram', tramDir)
trainingData('zebra', zebraDir)

#---------------------------------------------------------------------------------------

# Label encoding

numClasses = len(np.unique(labelList))

le = LabelEncoder() 

e_labelList = le.fit_transform(labelList) # transform the string labels to numeric values. It is a necessary step for the model
e_labelList = to_categorical(e_labelList, numClasses) # split the labels into 4 categories -- ONE HOT ENCODING
imgList = np.array(imgList)
imgList = imgList/255.0 # normalization [0,1] converting the images into numpy arrays

#-------------------------------------------------------------------------------------

# Train Validation split

X_train, X_valid, y_train, y_valid = train_test_split(imgList, e_labelList, test_size=0.20, random_state=42, shuffle=True)

# Train Test split

X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size=0.25, random_state=42,  shuffle=True)

#-----------------------------------------------------------------------------------

# A little data augmentation

aug_gen = ImageDataGenerator( 
        width_shift_range=0.1,  
        height_shift_range=0.1) 

aug_gen.fit(X_train)

# Showing the shape after splitting the dataset

print('')
print('Shape of object classes:', numClasses) # number of object classes
print('')
print('Shape of training images:', X_train.shape) # number of training samples
print('Shape of training labels:', y_train.shape) # number of training samples
print('Shape of validation images:', X_valid.shape) # number of testing samples
print('Shape of validation labels:', y_valid.shape) # number of testing labels
print('Shape of testing images:', X_test.shape) # number of testing samples
print('Shape of testing labels:', y_test.shape) # number of testing labels

#-------------------------------------------------------------------------------------

# Plot some random images

fig, axis = plt.subplots(5,2)
fig.set_size_inches(15,15)
for i in range(5):
  for j in range(2):
    randLabel = rn.randint(0, len(labelList))
    axis[i, j].imshow(imgList[randLabel])
    axis[i, j].set_title('Object: '+ labelList[randLabel])
    axis[i, j].grid(color='w', linestyle='-', linewidth=0.5)

plt.tight_layout()
plt.savefig('/content/mydrive/image_classifier/saved')
plt.show()

#-------------------------------------------------------------------------------------

# Transfer learning model (Feature extractor model)

base_model = VGG16(include_top=False, input_shape = (imgSize,imgSize,3), weights = None, pooling='avg')

base_model.load_weights(weights_path)

for i in range (len(base_model.layers)):
    print (i,base_model.layers[i])
  
for layer in base_model.layers[15:]:
    layer.trainable=True
for layer in base_model.layers[0:15]:
    layer.trainable=False

model = Sequential()
model.add(base_model)
model.add(GlobalAveragePooling2D())
model.add(Dropout(0.5))
model.add(Dense(5,activation='softmax'))
model.summary()

#--------------------------------------------------------------------------------------

# Transfer learning model (1 layer unforzen - Fine - tuning model)

base_model = VGG16(include_top=False, input_shape = (imgSize,imgSize,3), weights = None, pooling='avg')

base_model.load_weights(weights_path)

for layer in base_model.layers:
    layer.trainable = False
    
for layer in base_model.layers:
    print(layer,layer.trainable)

model = Sequential()
model.add(base_model)
model.add(GlobalAveragePooling2D())
model.add(Dropout(0.5))
model.add(Dense(5,activation='softmax'))
model.summary()

#-------------------------------------------------------------------------------------

# Scratch model

model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=(imgSize, imgSize, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.3))

model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(numClasses))
model.add(Activation('softmax'))
model.summary()

from keras.utils import plot_model

plot_model(model, to_file='/content/mydrive/image_classifier/saved/convolutional_neural_network.png')

#---------------------------------------------------------------------------------------

# Compile the model

model.compile(loss = 'categorical_crossentropy', optimizer=Adam(lr=1e-4) , metrics = ['accuracy'])

#--------------------------------------------------------------------------------------------

# Model callbacks

filepath = '/content/mydrive/image_classifier/weights/weights_vgg16_scratch2.h5'
checkpoint = ModelCheckpoint(filepath, monitor = 'val_acc', verbose = 1, save_best_only = True, mode ='max') # save only the best model
earlystop = EarlyStopping(monitor = 'val_loss', patience = 5, verbose = 1, mode ='min') # early model stop, which gives the best validation loss #  min_delta=0.001

checkpoint = ModelCheckpoint(
    filepath,
    monitor='val_loss',
    verbose=1,
    save_best_only=True,
    mode='min',
    save_weights_only=False,
    period=1
)
earlystop = EarlyStopping( # if the valdaiton loss not decreasing after  10 steps, to prevent over fitting we stop the trainign process
    monitor='val_loss',
    min_delta=0.001,
    patience=10,
    verbose=1,
    mode='auto'
)

callbackList = [checkpoint, earlystop]

#--------------------------------------------------------------------------------------------

num_train = X_train.shape[0]
num_test = X_test.shape[0]
num_valid = X_valid.shape[0]

# Model training

history = model.fit_generator(
    aug_gen.flow(X_train,y_train, batch_size=batchSize),
    validation_data  = (X_valid,y_valid),
    validation_steps = num_valid//batchSize,
    steps_per_epoch  = num_train//batchSize,
    epochs = 150, 
    verbose = 1,
    callbacks=callbackList)

print('Training complete')

#--------------------------------------------------------------------------------------------

# Model evaluate

finalHistory(history)

score = model.evaluate(X_test, y_test)

print('Model test loss:', score[0])
print('Model test accuracy:', score[1])
